In [1]:
!pip uninstall -y ipywidgets azureml-widgets
!pip install ipywidgets==7.6.5 azureml-widgets --no-cache-dir

Found existing installation: ipywidgets 7.0.0
Uninstalling ipywidgets-7.0.0:
  Successfully uninstalled ipywidgets-7.0.0
Found existing installation: azureml-widgets 1.60.0
Uninstalling azureml-widgets-1.60.0:
  Successfully uninstalled azureml-widgets-1.60.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.8/121.8 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 90.0 MB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 160.8 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.0.8
    Uninstalling widgetsnbextension-3.0.8:
      Successfully uninstalled widgetsnbextension-3.0.8


In [18]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-283741
Azure region: eastus2
Subscription id: 3e42d11f-d64d-4173-af9b-12ecaa1030b3
Resource group: aml-quickstarts-283741


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "my-project2-cluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

vm_size = "STANDARD_D2_V2"
max_nodes = 4
idle_seconds_before_scaledown = 120

try:
    aml_compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print(f"Found existing compute target: {cluster_name}")
except ComputeTargetException:
    print(f"Compute target '{cluster_name}' not found. Creating a new one...")

    compute_config = AmlCompute.provisioning_configuration(
        vm_size=vm_size,
        max_nodes=max_nodes,
        min_nodes=0,
        idle_seconds_before_scaledown=idle_seconds_before_scaledown
    )
    
    aml_compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
    print(f"Submitting compute creation request for '{cluster_name}'...")

    aml_compute_target.wait_for_completion(show_output=True, min_node_count=0, timeout_in_minutes=2)
    print(f"Compute cluster '{cluster_name}' provisioned successfully.")

print("\nCurrent status of the compute cluster:")
print(aml_compute_target.get_status().serialize())

Compute target 'my-project2-cluster' not found. Creating a new one...
Submitting compute creation request for 'my-project2-cluster'...
InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
Compute cluster 'my-project2-cluster' provisioned successfully.

Current status of the compute cluster:
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2025-06-18T16:08:10.744000+00:00', 'errors': None, 'creationTime': '2025-06-18T16:08:01.320049+00:00', 'modifiedTime': '2025-06-18T16:08:11.876873+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeSc

In [4]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
### YOUR CODE HERE ###
ps = RandomParameterSampling(
    {
        '--C': uniform(0.01, 100),
        '--max_iter': choice(50, 100, 200, 300)
    }
)

# Specify a Policy
### YOUR CODE HERE ###
policy = BanditPolicy(evaluation_interval=3, slack_factor=0.1)

# if "training" not in os.listdir():
#     os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
### YOUR CODE HERE ###
src = ScriptRunConfig(
    source_directory='./',
    script='train.py',
    environment=sklearn_env,
    compute_target='my-project2-cluster'
)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(
    run_config=src,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_concurrent_runs=4,
    max_total_runs=30
)

In [5]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

2025-06-17 23:08:15.346609: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750201695.372636   14113 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750201695.380523   14113 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750201695.400873   14113 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750201695.400922   14113 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750201695.400925   14113 computation_placer.cc:177] computation placer alr

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_5887ee5f-778e-4c11-badf-0dea10152875
Web View: https://ml.azure.com/runs/HD_5887ee5f-778e-4c11-badf-0dea10152875?wsid=/subscriptions/3e42d11f-d64d-4173-af9b-12ecaa1030b3/resourcegroups/aml-quickstarts-283693/workspaces/quick-starts-ws-283693&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2025-06-17T23:08:12.2131450Z][GENERATOR][DEBUG]Sampled 4 jobs from search space 
[2025-06-17T23:08:12.5383877Z][SCHEDULER][INFO]Scheduling job, id='HD_5887ee5f-778e-4c11-badf-0dea10152875_0' 
[2025-06-17T23:08:12.6405833Z][SCHEDULER][INFO]Scheduling job, id='HD_5887ee5f-778e-4c11-badf-0dea10152875_1' 
[2025-06-17T23:08:12.6425074Z][SCHEDULER][INFO]Scheduling job, id='HD_5887ee5f-778e-4c11-badf-0dea10152875_2' 
[2025-06-17T23:08:12.6416432Z][SCHEDULER][INFO]Scheduling job, id='HD_5887ee5f-778e-4c11-badf-0dea10152875_3' 
[2025-06-17T23:08:13.1060150Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_5887ee5f-778e-4c11-badf-0dea10152875_2' 
[2025-06-17T23:0

{'runId': 'HD_5887ee5f-778e-4c11-badf-0dea10152875',
 'target': 'my-project2-cluster',
 'status': 'Completed',
 'startTimeUtc': '2025-06-17T23:08:10.643404Z',
 'endTimeUtc': '2025-06-17T23:22:49.53066Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'ba5eee23-fe80-4a5d-945f-9678109fe183',
  'user_agent': 'python/3.10.11 (Linux-6.8.0-1026-azure-x86_64-with-glibc2.35) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.60.0',
  'best_child_run_id': 'HD_5887ee5f-778e-4c11-badf-0dea10152875_22',
  'score': '0.9089529590288316',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_5887ee5f-778e-4c11-badf-0dea10152875_22'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClientType': 'azure

In [6]:
import joblib
from azureml.core.model import Model
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n Parameters:',parameter_values)

model_file_name = 'hyperdriveBestModel.pkl'
remote_model_path = f'outputs/model.pkl'
local_model_path = os.path.join('./training', model_file_name)

print(f"Downloading model from run {best_run.id} to {local_model_path}...")
best_run.download_file(name=remote_model_path, output_file_path=local_model_path)
print("Model downloaded successfully.")

print(f"Registering model '{model_file_name}' from run {best_run.id}...")
model = Model.register(workspace=ws,
                       model_path=local_model_path, # Path to the downloaded model file
                       model_name="best_hyperdrive_model", # Name for your registered model
                       tags={'run_id': best_run.id, 'accuracy': best_run.get_metrics().get('Accuracy', 'N/A')},
                       description="Logistic Regression model tuned with HyperDrive")

print(f"Model registered successfully with name: {model.name}, version: {model.version}")

Best Run Id:  HD_5887ee5f-778e-4c11-badf-0dea10152875_22

 Accuracy: 0.9089529590288316

 Parameters: ['--C', '64.07956452127272', '--max_iter', '300']
Model downloaded successfully.
Registering model 'hyperdriveBestModel.pkl' from run HD_5887ee5f-778e-4c11-badf-0dea10152875_22...
Registering model best_hyperdrive_model
Model registered successfully with name: best_hyperdrive_model, version: 1


In [4]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv", validate=True, separator=',')

In [5]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


In [14]:
from azureml.train.automl import AutoMLConfig
from azureml.core import Dataset
import logging
import pandas as pd

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.

data = pd.concat([x, y], axis=1)

temp_csv_file = 'automl_temp_data.csv'
data.to_csv(temp_csv_file, index=False)
print(f"Temporary CSV created at: {temp_csv_file}")

default_ds = ws.get_default_datastore()
datastore_target_directory = 'cleaned_data_for_automl'
datastore_target_path_full = os.path.join(datastore_target_directory, temp_csv_file)
print(f"Uploading '{temp_csv_file}' to datastore at '{default_ds.name}/{datastore_target_path_full}'...")
default_ds.upload_files(files=[temp_csv_file],
                        target_path=datastore_target_directory, # Only the directory part here
                        overwrite=True,
                        show_progress=True)
print("File uploaded to datastore.")

training_data = Dataset.Tabular.from_delimited_files(path=[(default_ds, datastore_target_path_full)], validate=False, separator=',')

automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=training_data,
    label_column_name='y',
    iterations=30,
    compute_target='my-project2-cluster',
    enable_early_stopping=True,
    max_concurrent_iterations=4,
    featurization='auto',
    verbosity=logging.INFO,
    debug_log='automl_errors.log',
    enable_onnx_compatible_models=True,
    n_cross_validations=5)

Temporary CSV created at: automl_temp_data.csv
Uploading 'automl_temp_data.csv' to datastore at 'workspaceblobstore/cleaned_data_for_automl/automl_temp_data.csv'...
Uploading an estimated of 1 files
Uploading automl_temp_data.csv
Uploaded automl_temp_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files
File uploaded to datastore.


In [15]:
# Submit your automl run
from azureml.widgets import RunDetails

### YOUR CODE HERE ###
automl_run = exp.submit(config=automl_config, show_output=True)
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

Submitting remote run.
No run_configuration provided, running on my-project2-cluster with default configuration
Running on remote compute: my-project2-cluster


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_d927554f-272a-439c-874d-5a3aae95bdf4,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+-----

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_d927554f-272a-439c-874d-5a3aae95bdf4,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Number of samples in the training data|
+==============================+================================+======================================+
|3692                          |True                            |32950                                 |
+------------------------------+--------------------------------+----

{'runId': 'AutoML_d927554f-272a-439c-874d-5a3aae95bdf4',
 'target': 'my-project2-cluster',
 'status': 'Completed',
 'startTimeUtc': '2025-06-18T17:20:43.256111Z',
 'endTimeUtc': '2025-06-18T18:01:33.087749Z',
 'services': {},
 'properties': {'num_iterations': '30',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'my-project2-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"3e42d11f-d64d-4173-af9b-12ecaa1030b3","resource_group":"aml-quickstarts-283741","workspace_name":"quick-starts-ws-283741","region":"eastus2","compute_target":"my-project2-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":true,"enable_parallel_run":false,"num_procs":null,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"

In [16]:
# Retrieve and save your best automl model.
from azureml.core.model import Model

### YOUR CODE HERE ###
best_run, fitted_model = automl_run.get_output()

print(f"Best AutoML run ID: {best_run.id}")
print(f"Best model pipeline type: {type(fitted_model)}")

# Get metrics from the best run
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['accuracy'])
print('\n Parameters:',parameter_values)

model_file_name = 'automlBestModel.pkl'
remote_model_path = f'outputs/model.pkl'
local_model_path = os.path.join('./training', model_file_name)

print(f"Downloading model from run {best_run.id} to {local_model_path}...")
best_run.download_file(name=remote_model_path, output_file_path=local_model_path)
print("Model downloaded successfully.")

print(f"Registering model '{model_file_name}' from run {best_run.id}...")
model = Model.register(workspace=ws,
                       model_path=local_model_path, # Path to the downloaded model file
                       model_name="best_automl_model", # Name for your registered model
                       tags={'run_id': best_run.id, 'accuracy': best_run.get_metrics().get('accuracy', 'N/A')},
                       description="Best model tuned with automl")

print(f"Model registered successfully with name: {model.name}, version: {model.version}")

Best AutoML run ID: AutoML_d927554f-272a-439c-874d-5a3aae95bdf4_29
Best model pipeline type: <class 'azureml.training.tabular.models.pipeline_with_ytransformations.PipelineWithYTransformations'>
Best Run Id:  AutoML_d927554f-272a-439c-874d-5a3aae95bdf4_29

 Accuracy: 0.9170864946889224

 Parameters: []
Model downloaded successfully.
Registering model 'automlBestModel.pkl' from run AutoML_d927554f-272a-439c-874d-5a3aae95bdf4_29...
Registering model best_automl_model
Model registered successfully with name: best_automl_model, version: 2


In [19]:
# Delete Compute Target

cluster_name = "my-project2-cluster"
aml_compute_target = ComputeTarget(workspace=ws, name=cluster_name)
aml_compute_target.delete()